# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2021-08-20 18:47:03.917689: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-20 18:47:03.917715: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf.__version__

'2.6.0'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [9]:
ann = tf.keras.models.Sequential()

2021-08-20 18:47:07.848236: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-20 18:47:07.849650: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-20 18:47:07.849757: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-08-20 18:47:07.849804: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-08-20 18:47:07.849851: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

### Adding the input layer and the first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units=6, activation='tanh'))

### Adding the second hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [12]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [14]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

2021-08-20 18:47:08.307925: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
250/250 [==============================] - 1s 675us/step - loss: 0.5905 - accuracy: 0.6811
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4482 - accuracy: 0.8036
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4224 - accuracy: 0.8166
Epoch 4/100
250/250 [==============================] - 0s 723us/step - loss: 0.4091 - accuracy: 0.8215
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3979 - accuracy: 0.8256
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3862 - accuracy: 0.8356
Epoch 7/100
250/250 [==============================] - 0s 960us/step - loss: 0.3769 - accuracy: 0.8432
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3697 - accuracy: 0.8472
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3648 - accuracy: 0.8481
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3604 - 

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [15]:
y_pred = ann.predict(X_test)
y_pred

array([[0.24794072],
       [0.360164  ],
       [0.12352729],
       ...,
       [0.14544854],
       [0.1191549 ],
       [0.21078384]], dtype=float32)

In [16]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1519   76]
 [ 200  205]]


---
# Avaliando o Modelo

In [18]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [20]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

2021-08-20 18:47:43.283879: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-20 18:47:43.284228: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-20 18:47:43.352273: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-20 18:47:43.352311: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-20 18:47:43.412168: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/10
Epoch 1/10
Epoch 1/10Epoch 1/10

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
720/720 [==============================] - 4s 5ms/step - loss: 0.4998 - accuracy: 0.7951
Epoch 2/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4839 - accuracy: 0.7943
Epoch 2/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4840 - accuracy: 0.7975
Epoch 2/10
Epoch 2/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4918 - accuracy: 0.7957
Epoch 2/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4949 - accuracy: 0.7964
Epoch 2/10
720/720 [==============================] - 3s 5ms/step - loss: 0.4292 - accuracy: 0.7942
Epoch 3/10
720/720 [==============================] - 3s 5ms/step - loss: 0.4291 - accuracy: 0.7992
Epoch 3/10
659/720 [==========================>...] - ETA: 0s - loss: 0.4351 - accuracy: 0.7924720/720 [==============================] - 3s 5ms/step - loss: 0.4270 - accuracy: 0.8008
Epoch 3/10
720/720 [========

In [21]:
print(mean,variance)

0.8387499988079071 0.014705438292060806


# Melhorando o modelo

In [22]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [23]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

Epoch 1/10Epoch 1/10

Epoch 1/10
Epoch 1/10Epoch 1/10

Epoch 1/10
Epoch 1/10
Epoch 1/10
720/720 [==============================] - 3s 2ms/step - loss: 0.5090 - accuracy: 0.7922
Epoch 2/10
720/720 [==============================] - 3s 2ms/step - loss: 0.5247 - accuracy: 0.7964
Epoch 2/10
720/720 [==============================] - 3s 2ms/step - loss: 0.4831 - accuracy: 0.7944
Epoch 2/10
720/720 [==============================] - 3s 3ms/step - loss: 0.4809 - accuracy: 0.7956
Epoch 2/10
720/720 [==============================]Epoch 2/10 - 3s 3ms/step - loss: 0.4950 - accuracy: 0.7967
Epoch 2/10

720/720 [==============================] - 3s 3ms/step - loss: 0.4865 - accuracy: 0.7960
Epoch 2/10
707/720 [============================>.] - ETA: 0s - loss: 0.4497 - accuracy: 0.8033 - 2s 2ms/step - loss: 0.4328 - accuracy: 0.7971
Epoch 3/10
720/720 [==============================] - 2s 3ms/step - loss: 0.4506 - accuracy: 0.8033
Epoch 3/10
720/720 [==============================] - 2s 2ms/step - 

In [24]:
print(mean,variance)

0.8209999978542328 0.015049916056153216


# Tuning the ANN

In [25]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [26]:
%%time

grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
720/720 [==============================] - 1s 611us/step - loss: 0.4841 - accuracy: 0.7965
Epoch 2/50
720/720 [==============================] - 0s 615us/step - loss: 0.4314 - accuracy: 0.7982
Epoch 3/50
720/720 [==============================] - 1s 972us/step - loss: 0.4319 - accuracy: 0.7997
Epoch 4/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4322 - accuracy: 0.8050
Epoch 5/50
720/720 [==============================] - 1s 979us/step - loss: 0.4300 - accuracy: 0.8085
Epoch 6/50
720/720 [==============================] - 0s 589us/step - loss: 0.4299 - accuracy: 0.8112
Epoch 7/50
720/720 [==============================] - 0s 599us/step - loss: 0.4238 - accuracy: 0.8183
Epoch 8/50
720/720 [==============================] - 1s 755us/step - loss: 0.4188 - accuracy: 0.8196
Epoch 9/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4126 - accuracy: 0.8225
Epoch 10/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4068 - ac

In [27]:
print('Best params:\n',grid_search.best_params_,'\nBest accuracy:\n',grid_search.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'adam'} 
Best accuracy:
 0.859125


# Halving Grid Search

In [28]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

In [29]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

2021-08-20 22:26:34.585132: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-20 22:26:34.585133: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-20 22:26:34.585139: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-20 22:26:34.585137: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-20 22:26:34.585137: W tensorflow/stream_executor/platform/default/dso_lo

Epoch 1/50Epoch 1/50Epoch 1/50

Epoch 1/50

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
80/80 [==============================] - 2s 4ms/step - loss: 0.6804 - accuracy: 0.7860
Epoch 2/50
80/80 [==============================] - 2s 4ms/step - loss: 0.6773 - accuracy: 0.7810
Epoch 2/50
80/80 [==============================] - 2s 6ms/step - loss: 0.6757 - accuracy: 0.7985
Epoch 2/50
80/80 [==============================] - 2s 6ms/step - loss: 0.6780 - accuracy: 0.7935
Epoch 2/50
80/80 [==============================] - 0s 4ms/step - loss: 0.6282 - accuracy: 0.7910
Epoch 3/50
 1/80 [..............................] - 0s 4ms/step - loss: 0.5922 - accuracy: 0.7822
 - ETA: 0s - loss: 0.5774 - accuracy: 0.8000Epoch 3/50
80/80 [==============================] - 0s 3ms/step - loss: 0.6031 - accuracy: 0.7947
Epoch 3/50
80/80 [==============================] - 0s 4ms/step - loss: 0.6275 - accuracy: 0.8010
Epoch 3/50
65/80 [=======================>......] - 0s 4ms/step - loss: 0.6081 - accuracy: 0.79

In [ ]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'rmsprop'} 
Best accuracy:
 0.8346683354192741


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)